In [1]:
import time
import pandas as pd
import numpy as np
import os

In [2]:
def deco_print(line):
	print('>==================> ' + line)

path= 'C:/Users/zihanlin/Desktop/File//Research/cleaned_data/Return.csv'
# path = '../../datasets/Weber/NpForMartinLettau.csv'
deco_print('Loading data from ' + path)
df = pd.read_csv(path, header= None)
deco_print('Loading finished!')
path_mom= 'C:/Users/zihanlin/Desktop/File/Research/cleaned_data/MOMclean.csv'
deco_print('Loading data from ' + path_mom)
df_mom = pd.read_csv(path_mom, header= None)
deco_print('Loading finished!')
path_lme= 'C:/Users/zihanlin/Desktop/File/Research/cleaned_data/Sizeclean.csv'
deco_print('Loading data from' + path_lme)
df_lme = pd.read_csv(path_lme, header= None)
deco_print('Loading finished!')
path_IdioVol= 'C:/Users/zihanlin/Desktop/File/Research/cleaned_data/vol.csv'
deco_print('Loading data from' + path_IdioVol)
df_IdioVol = pd.read_csv(path_IdioVol, header= None)
deco_print('Loading finished!')
path_beme= 'C:/Users/zihanlin/Desktop/File/Research/cleaned_data/BMclean.csv'
deco_print('Loading data from' + path_beme)
df_beme = pd.read_csv(path_beme, header=None)
deco_print('Loading finished!')
# A larger portfolio
path_beme= 'C:/Users/zihanlin/Desktop/File/Research/cleaned_data/Bookclean.csv'
deco_print('Loading data from' + path_beme)
df_book = pd.read_csv(path_beme, header=None)
deco_print('Loading finished!')

>==================> Loading data from C:/Users/zihanlin/Desktop/File//Research/cleaned_data/Return.csv
>==================> Loading finished!
>==================> Loading data from C:/Users/zihanlin/Desktop/File/Research/cleaned_data/MOMclean.csv
>==================> Loading finished!
>==================> Loading data fromC:/Users/zihanlin/Desktop/File/Research/cleaned_data/Sizeclean.csv
>==================> Loading finished!
>==================> Loading data fromC:/Users/zihanlin/Desktop/File/Research/cleaned_data/vol.csv
>==================> Loading finished!
>==================> Loading data fromC:/Users/zihanlin/Desktop/File/Research/cleaned_data/BMclean.csv
>==================> Loading finished!
>==================> Loading data fromC:/Users/zihanlin/Desktop/File/Research/cleaned_data/Bookclean.csv
>==================> Loading finished!


In [3]:
returns= pd.DataFrame.as_matrix(df)
momentum= pd.DataFrame.as_matrix(df_mom)
size= pd.DataFrame.as_matrix(df_lme)
Idio_vol= pd.DataFrame.as_matrix(df_IdioVol)
beme= pd.DataFrame.as_matrix(df_beme)
# For book data.
book= pd.DataFrame.as_matrix(df_book)
fama_french_data= np.load('C:\\Users\\zihanlin\\Desktop\\File\\Research\\npz_data\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']
r_f= fama_french_data[:,4]
fama_french_date= fama_french_data[:,0]
r_f= r_f/100
date_start= np.asscalar(np.argwhere(fama_french_date==198401))
date_end= np.asscalar(np.argwhere(fama_french_date==201401))
returns= returns-r_f[date_start:date_end:,np.newaxis]

In [4]:
chara_data= np.stack((momentum, size, Idio_vol, beme, book))
# chara_data= np.stack((size, beme))
chara_data= np.transpose(chara_data, (1,2,0))
# returns= returns[100:319,:]
# chara_data= chara_data[100:319, :]

In [5]:
T, N= returns.shape
def get_tree(depth, res, nodetag,  value_array, L, d, chara_data, oldlist, parent):
    if depth == d:
        final_array = np.ones([N, 1])
        for k in range(L):
            temp = value_array[:, k].reshape([N, 1])
            final_array = final_array * temp
        # print(np.sum(final_array))
        if np.sum(final_array) < 10 ** (-3):
            print('An error over here')
        final_array = 1.0 * final_array / np.sum(final_array)
        # It alrady gets updated.
        # if np.sum(final_array)<10**(-3):
        #     print (value_array)
        res.append([final_array]) 
        nodetag.append(oldlist)
    else:
        for i in range(L):
            for j in range(2):
                if len(parent) != 0:
                    index = np.ones((N), dtype=bool)
                    for k in parent:
                        index = (index) & ((value_array[:, k] != 0))
                    data_median = np.median(chara_data[i, index])
                else:
                    index = (value_array[:, i] != 0)
                    data_median = np.median(chara_data[i, index])
                temp_array = value_array.copy()
                newlist= list(oldlist)
                newlist+=[(i,j)]
                parent.append(i)
                if j == 0:
                    temp_array[(index) & (chara_data[i, :] <= data_median), i] = 0
                    get_tree(depth + 1, res, nodetag, temp_array, L, d, chara_data, newlist, parent)
                elif j == 1:
                    temp_array[(index) & (chara_data[i, :] >= data_median), i] = 0
                    get_tree(depth + 1, res, nodetag, temp_array, L, d, chara_data, newlist, parent)
                parent.pop()
L= chara_data.shape[2]
d= 4
portfolio = np.zeros((T - 1, (2*L)**d))
for i in range(T-1):
# for i in range(0,1):
    if i%20==0:
            print ('Have gone through %d' %(i))
    # print ('Have gone through %d' %(i))
    res = []
    nodetag= []
# The first parameter is number of characteristics, and the second is the depth of the tree.
    get_tree(0, res, nodetag, np.ones([N, L]), L, d, chara_data[i,:].T, [], [])
    # output = np.array(res[0][0])
    # output = np.reshape(output, [N, 1])
    # for j in range(1, len(res)):
    #     output = np.append(output, np.array(res[j][0]), axis=1)
    # portfolio[i, :] = returns[i + 1, :].dot(output)
    for j in range(len(res)):
        portfolio[i][j]= returns[i+1,:].dot(res[j])

Have gone through 0
Have gone through 20
Have gone through 40
Have gone through 60
Have gone through 80
Have gone through 100
Have gone through 120
Have gone through 140
Have gone through 160
Have gone through 180
Have gone through 200
Have gone through 220
Have gone through 240
Have gone through 260
Have gone through 280
Have gone through 300
Have gone through 320
Have gone through 340


In [2]:
import scipy.linalg as la
# This K is different from that K.
def insample_sharpe(K, portfolio, RP= True, Corr= True):    
    gamma =10
    M= portfolio.shape[1]
    if RP:
        covmat = portfolio.T.dot(portfolio) 
        portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
        covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
    else:
        covmat = np.cov(portfolio.T)
    if Corr:
        covmat_t_diag= np.diag(1./np.sqrt(np.diag(covmat_t)))
        covmat_t= covmat_t_diag.dot(covmat_t.dot(covmat_t_diag))
    variance, loading= la.eigh(covmat, eigvals=(M-K, M-1))
    Factor= portfolio.dot(loading)
    output= []
# Factor = portfolio.dot(loading_chosen)/N
    for i in range(K-1,-1,-1):
        mu = np.mean(Factor[:,i:K], axis=0)
        covmat_factor = np.cov(Factor[:,i:K].T)
        if i==(K-1):
            output.append (abs(mu)*covmat_factor**(-1/2))
        else:
            output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
    twodecimals = ["&%.2f" % np.real(v) for v in output]
    print (''.join(twodecimals))

In [6]:
# data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/tree_portfolio.npz')
data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/tree_portfolio.npz')
portfolio= data['portfolio']
def compute_sharpe(K,portfolio, RP= True, Corr= True, Time= True, delta= 900, Shrink= False):
    import scipy.linalg as la
    T, M = portfolio.shape
    window= 120
    optimal_return = np.zeros((T-window))
    gamma = 10    
    ones_T= np.ones((window, 1))
    for t in range(T-window):
        portfolio_t = portfolio[t:(t + window), :]
        if Time: 
            if RP:
                covmat_t= (np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window))).dot(portfolio_t)
                covmat_t= (covmat_t.dot(portfolio_t.T)).dot(np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window)))       
            else: 
                covmat_t= np.cov(portfolio_t)        
        else:
            if RP:
                covmat_t = portfolio_t.T.dot(portfolio_t) 
                portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
                covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t)/window * gamma
            else:
                covmat_t= np.cov(portfolio_t.T)
        if Corr:
            covmat_t_diag= np.diag(1./np.sqrt(np.diag(covmat_t)))
            covmat_t= covmat_t_diag.dot(covmat_t.dot(covmat_t_diag))
        # Get the correlation matrix
        if Time:
            variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
            Factor_t= Factor_t[:,:K]
            loading_t= Factor_t.T.dot(portfolio_t)
            loading_t= loading_t.T
        else:
            variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
            Factor_t= portfolio_t.dot(loading_t)
        if Shrink:
            for i in range(K):
                threshold= np.partition(abs(loading_t[:,i]),-delta)[-delta]
                loading_t[np.abs(loading_t[:,i])<threshold,i]=0
                loading_t[:,i]= loading_t[:,i]/np.linalg.norm(loading_t[:,i])
                
        mu= np.mean(Factor_t, axis= 0)
        covmat_factor= np.cov(Factor_t.T)
        if K>=2:
            weight= np.linalg.inv(covmat_factor).dot(mu)
        else:
            weight= mu/covmat_factor
        if Time:        
            optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(np.linalg.inv(loading_t.T.dot(loading_t))).dot(weight)
        else:
            optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(np.linalg.inv(loading_t.T.dot(loading_t))).dot(weight)
        # optimal_return[t]= Factor_t[t,:].dot(weight)
    return optimal_return
def easy_sharpe(optimal_return):
        mean_optimal= np.mean(optimal_return)
        cov_optimal= np.cov(optimal_return)
        return abs(mean_optimal)/np.sqrt(cov_optimal)
K= 20
optimal_sharpe= np.zeros((K))
for i in range(K):
    optimal_sharpe[i]= easy_sharpe(compute_sharpe(i+1,portfolio))
print (optimal_sharpe)

[ 0.22807927  0.23464113  0.24974003  0.4266267   0.46040466  0.45389013
  0.44943252  0.45367517  0.45632729  0.4812883   0.48166277  0.48316771
  0.48247971  0.48081463  0.49345735  0.4788871   0.47411505  0.4725792
  0.46797566  0.46254032]


In [8]:
# Get the alpha
import scipy.linalg as la
import numpy as np
def getalpha(K, portfolio, RP= True, Corr= True, Time= True, delta= 600, Shrink= True ):
    window= 120
    gamma = 100
    T, M = portfolio.shape
    pricing_error= np.zeros((T-window,M ))
    ones_T= np.ones((window, 1))
    for t in range(T-window):
#         if t % 20 == 0:
#             print('Have gone through iteration %d' % (t))
#         print (t)
        portfolio_t = portfolio[t:(t + window), :]
        if Time: 
            if RP:
                covmat_t= (np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window))).dot(portfolio_t)
                covmat_t= (covmat_t.dot(portfolio_t.T)).dot(np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window)))       
            else: 
                covmat_t= np.cov(portfolio_t)        
        else:
            if RP:
                covmat_t = portfolio_t.T.dot(portfolio_t) 
                portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
                covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t)/window * gamma
            else:
                covmat_t= np.cov(portfolio_t.T)
        if Corr:
            covmat_t_diag= np.diag(1./np.sqrt(np.diag(covmat_t)))
            covmat_t= covmat_t_diag.dot(covmat_t.dot(covmat_t_diag))
#         print (np.isinf(covmat_t))
#         print (np.isnan(covmat_t))
        # Get the correlation matrix
        if Time:
            variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
            Factor_t= Factor_t[:,:K]
            loading_t= Factor_t.T.dot(portfolio_t)
            loading_t= loading_t.T
        else:
            variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
            Factor_t= portfolio_t.dot(loading_t)
        if Shrink:
            for i in range(K):
                threshold= np.partition(abs(loading_t[:,i]),-delta)[-delta]
                loading_t[np.abs(loading_t[:,i])<threshold,i]=0
                loading_t[:,i]= loading_t[:,i]/np.linalg.norm(loading_t[:,i])
        pricing_error[t,:] = (portfolio[t + window, :]- portfolio[t+window, :].dot(loading_t).dot(np.linalg.inv(loading_t.T.dot(loading_t))).dot(loading_t.T))
    avg_pricing_error= np.mean(pricing_error, axis= 0)
    print (np.sqrt(np.sum(avg_pricing_error**2)/M))
data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/tree_portfolio.npz')
portfolio= data['portfolio']
print (portfolio.shape)
portfolio= portfolio*100
getalpha(3, portfolio, RP= True, Corr= True, Time= True, delta= 10, Shrink= False)

(359L, 4096L)
0.224858632909


In [42]:
T, N= returns.shape
L= chara_data.shape[2]
D= 10
portfolio= np.zeros((T-1, L*D))
for i in range(T-1):
    for j in range(L):
        chara_data_t= chara_data[i,:,j]
        index= np.argsort(chara_data_t)
        length= int(len(index)/D)
        for k in range(D):
            index_k= index[int(length*k):int(length*(k+1))]
            portfolio[i, j*D+k]= np.mean(returns[i+1,index_k ])

In [13]:
portfolio_data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/large_tree_portfolio.npz')
portfolio= portfolio_data['portfolio']
nodetag= portfolio_data['nodetag']


In [12]:
path_target = 'C:/Users/zihanlin/Desktop/File/Research/npz_data/large_tree_portfolio.npz'
print (nodetag.shape)
print('Saving data to ' + path_target)
np.savez(path_target, portfolio=portfolio, nodetag= nodetag)
print('Finished!')

(10000L, 4L, 2L)
Saving data to C:/Users/zihanlin/Desktop/File/Research/npz_data/large_tree_portfolio.npz
Finished!


In [33]:
import numpy as np
data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/tree_portfolio.npz')
portfolio_tree= data['portfolio']
data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/fama_french_portfolio.npz')
portfolio_decile= data['portfolio']
optimal_tree= compute_sharpe(4, portfolio_tree, True, True, True)
optimal_decile= compute_sharpe(3, portfolio_decile, True, True, True)
optimal_tree_pca= compute_sharpe(4, portfolio_tree, False, True, True)
optimal_decile_pca= compute_sharpe(3, portfolio_decile, False, True, False)
cum_ret_tree= np.cumsum(optimal_tree)
cum_ret_decile= np.cumsum(optimal_decile)
cum_ret_pca= np.cumsum(optimal_decile_pca)
cum_ret_tree_pca= np.cumsum(optimal_tree_pca)
import matplotlib.pyplot as plt
plt.plot(cum_ret_tree, label= 'Tree RPPCA')
plt.plot(cum_ret_decile,label= 'Fama-French RPPCA')
plt.plot(cum_ret_pca, label= 'Fama-French PCA')
plt.plot(cum_ret_tree_pca, label= 'Tree PCA')
plt.xlabel('Time')
plt.ylabel('Cumulative Return')
plt.legend(loc= 2)
plt.savefig('C:/Users/zihanlin/Desktop/File/Research/npz_data/Cum_ret.png')
plt.clf()

In [5]:
# Fama French Estimation
size= chara_data[:,:,1]
beme= chara_data[:,:,3]
T,N= returns.shape
M = 5
portfolio = np.zeros((T-1,25))
for i in range(T-1):
# for i in range(0,15):
    return_t= returns[i+1,:]
    basis= np.zeros((N, M*M))
    for j in range(M):
        for k in range(M):
            quantile_size_lower= np.percentile(size[i,:], int(100.0/M*(j)))
            quantile_size_upper= np.percentile(size[i,:], int(100.0/M*(j+1)))
            quantile_beme_lower= np.percentile(beme[i,:], int(100.0/M*(k)))
            quantile_beme_upper = np.percentile(beme[i,:], int(100.0/M*(k+1)))
            index= (size[i,:]>= quantile_size_lower)& (size[i,:]<= quantile_size_upper)& (beme[i,:]>= quantile_beme_lower)& (beme[i,:]<= quantile_beme_upper)
            portfolio[i, j*M+k]= np.mean(return_t[index])
path_target = 'C:/Users/zihanlin/Desktop/File/Research/npz_data/fama_french_portfolio.npz'
print('Saving data to ' + path_target)
np.savez(path_target, portfolio=portfolio)
print('Finished! ')

Saving data to C:/Users/zihanlin/Desktop/File/Research/npz_data/fama_french_portfolio.npz
Finished! 


In [46]:
delta= 10
K= 4
loading_chosen = loading[:, :K].copy()
loading_2_K= loading_chosen[:,:K].copy()
count= 0
for i in range(K):
    print ("This is the %d factor" % i)
    threshold= np.partition(abs(loading_2_K[:,i]),-delta)[-delta]
    loading_2_K[np.abs(loading_2_K[:,i])<threshold,i]=0
    loading_2_K[:,i]= loading_2_K[:,i]/np.linalg.norm(loading_2_K[:,i])*np.sqrt(N)
    index_K= np.argwhere(np.abs(loading_2_K[:,i])>=threshold)
    # print (index_K)
    for j in range(len(index_K)):
        beautiful_print(nodetag[index_K[j][0]])
        count+= 1    
Factor_sparse = portfolio.dot(loading_2_K)/N
rho= np.trace(np.linalg.inv(Factor.T.dot(Factor)/T).dot(Factor.T.dot(Factor_sparse)/T).dot(np.linalg.inv(Factor_sparse.T.dot(Factor_sparse)/T)).dot(Factor_sparse.T.dot(Factor)/T))
print (rho)

This is the 0 factor
[('momentum', 'small'), ('beme', 'big'), ('Idio_vol', 'small'), ('size', 'big')]
[('momentum', 'big'), ('Idio_vol', 'small'), ('size', 'big'), ('size', 'small')]
[('momentum', 'big'), ('beme', 'small'), ('momentum', 'big'), ('beme', 'small')]
[('momentum', 'big'), ('beme', 'small'), ('momentum', 'big'), ('beme', 'big')]
[('momentum', 'big'), ('beme', 'small'), ('beme', 'big'), ('momentum', 'big')]
[('size', 'big'), ('size', 'big'), ('size', 'small'), ('Idio_vol', 'small')]
[('size', 'big'), ('beme', 'small'), ('momentum', 'big'), ('beme', 'big')]
[('size', 'big'), ('beme', 'small'), ('size', 'big'), ('size', 'big')]
[('size', 'big'), ('beme', 'small'), ('beme', 'big'), ('momentum', 'big')]
[('beme', 'small'), ('momentum', 'big'), ('momentum', 'big'), ('beme', 'big')]
This is the 1 factor
[('size', 'big'), ('beme', 'big'), ('beme', 'big'), ('beme', 'big')]
[('Idio_vol', 'small'), ('Idio_vol', 'small'), ('beme', 'big'), ('beme', 'small')]
[('beme', 'big'), ('size', '

In [45]:
def beautiful_print(alist):
    char_list= ['momentum', 'size', 'Idio_vol', 'beme','Book']
    size_list= ['small', 'big']
    out_list= []
    for char in alist:
        out_list.append((char_list[char[0]], size_list[char[1]]))
    print (out_list)    

In [11]:
import pandas as pd
# data= pd.read_csv('C:/Users/zihanlin/Desktop/File/Research/25_Portfolios_ME_AC_5x5.csv')
# size_accrual= pd.DataFrame.as_matrix(data)
# size_accrual= size_accrual[0:606,1:]
data= pd.read_csv('C:/Users/zihanlin/Desktop/File/Research/decile.csv', header= 1)
portfolio= pd.DataFrame.as_matrix(data)
# print (np.sum(np.isnan(portfolio)))
# data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/decile_portfolio.npz')
# data= np.load('C:/Users/zihanlin/Desktop/File/Research/npz_data/tree_portfolio.npz')
# portfolio= data['portfolio']
portfolio= portfolio*100
portfolio= portfolio[0:636,:]
print (portfolio.shape)
getalpha(4, portfolio, RP= True, Corr= False, Time= True, delta= 600, Shrink= False)

(636L, 370L)
0
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
1
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., Fal

29
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
30
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False Fa

49
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
50
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False Fa

 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
78
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
79
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 

108
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
109
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False 

...

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
371
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False Fals

 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
416
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
417
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]

448
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
449
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False 

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
483
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False Fals

506
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
507
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False 